<a href="https://colab.research.google.com/github/inigmat/exupery/blob/main/df2png.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing necessary packages for creating PNG files from tables. Initial run takes about 1 min
!pip freeze | grep texlive || (apt-get update && apt-get install -y texlive texlive-latex-extra texlive-fonts-recommended dvipng)
!pip freeze | grep imagemagick || apt-get install -y imagemagick
!pip freeze | grep pdf2svg || apt-get install -y pdf2svg
!pip freeze | grep cairosvg || pip install cairosvg

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 2s (145 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
dvipng is already the newest version (1.15-1

In [2]:
'''
This code defines a function to convert a LaTeX table into a PNG image.

Function:
latex_table_to_png(obj, name)
- Converts a LaTeX table code into a PNG image.
- Parameters:
    - obj: The LaTeX table code.
    - name: The desired name for the output PNG file.
- Steps:
    1. Save the LaTeX code to a file named 'table.tex'.
    2. Compile the LaTeX file into a DVI file using the 'latex' command.
    3. Convert the DVI file to an SVG file using the 'dvisvgm' command.
    4. Convert the SVG file to a PNG file using the 'cairosvg' library.
    5. Download the PNG file using the 'files.download' function from Google Colab.

Note:
- The function assumes that the necessary packages and dependencies (such as 'latex', 'dvisvgm', and 'cairosvg') are installed and accessible in the environment.
- If any errors occur during the execution, appropriate error messages will be displayed.
'''

from google.colab import files
import subprocess
import cairosvg


def latex_table_to_png(obj, name):
    # LaTeX table code
    template = r'''\documentclass[preview]{{standalone}}
    \usepackage{{booktabs}}
    \begin{{document}}
    {}
    \end{{document}}
    '''

    # Save LaTeX code to a file
    with open('table.tex', 'w') as f:
        f.write(template.format(obj))

    try:
         # Compile LaTeX file to DVI file
        subprocess.check_call(['latex', 'table.tex'])

        # Convert DVI file to SVG file
        subprocess.check_call(['dvisvgm', '--no-fonts', 'table.dvi', '-o', 'table.svg'])

        # PNG file name
        png_filename = f'{name}.png'

        # Convert SVG file to PNG file
        cairosvg.svg2png(url='table.svg', write_to=png_filename, scale=1.7)

        # Download the file
        files.download(png_filename)

    except subprocess.CalledProcessError as e:
        print("Error executing command:", e)

    except IOError as e:
        print("Error with file operations:", e)


In [3]:
# Example
import pandas as pd

CSV_URL = 'https://raw.githubusercontent.com/inigmat/exupery/main/files/bbs.csv'
data = pd.read_csv(CSV_URL)
bbs = data.drop(columns='index')
bbs_latex = bbs.style.to_latex(hrules=True)
latex_table_to_png(bbs_latex, 'bbs')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Таблица результатов раскроя арматуры

In [4]:
%pip install binpacking

# Check the csv data

CSV_URL = 'https://raw.githubusercontent.com/inigmat/exupery/main/files/bbs.csv'

  Preparing metadata (setup.py) ... done
  Created wheel for binpacking: filename=binpacking-1.5.2-py3-none-any.whl size=10093 sha256=538a1fb595789f3cd6fb7d3789c956f792bf1a1342d678b512184f82c3cf1a6b
  Stored in directory: /root/.cache/pip/wheels/4f/09/07/93d7c3a8acc3f39fc972dd12b8b8131fdd00f9e61ca09ed723
Successfully built binpacking


In [5]:
import pandas as pd
from binpacking import to_constant_volume

# Constants
BAR_LENGTH = 11700
CSV_URL = CSV_URL


def calculate_required_bars(bar_schedule: pd.DataFrame, bar_length: int) -> tuple:
    """
    Calculates the cutting schemes based on input data (bar schedule)
    containing the length and quantity of bars and the length of the bars in stock.
    """
    items = []
    # Extracting data from DataFrame
    for length, quantity in zip(bar_schedule['Length'], bar_schedule['Qty']):
        items.extend([length] * quantity)
    # Calculating the cutting schemes using bin packing algorithm
    cutting_scheme = to_constant_volume(items, bar_length)
    return cutting_scheme


def generate_cutting_table(cutting_data: dict) -> pd.DataFrame:
    """
    Generates a dataframe (cutting schedule) with quantity,
    cuts, utilization, scrap, and diameter columns
    based on the cutting data (cutting schemes).
    """
    cutting_schedule = pd.DataFrame(columns=["Qty", "Cuts", "Utilization", "Scrap", "Dia"])
    # Processing cutting data for each diameter
    for dia, cutting_schemes in cutting_data.items():
        df_temp = pd.DataFrame(zip(cutting_schemes), columns=["Cuts"])
        # Calculate the quantity of each cutting scheme
        df_temp["Qty"] = df_temp["Cuts"].apply(lambda x: f"{cutting_schemes.count(x)}x")
        # Calculate the total length used by each cutting scheme
        df_temp["Utilization"] = df_temp["Cuts"].apply(sum)
        # Calculate the amount of scrap for each cutting scheme
        df_temp["Scrap"] = BAR_LENGTH - df_temp["Utilization"]
        # Add the diameter column
        df_temp["Dia"] = dia
        # Change the type of values in the Cuts column to drop duplicates
        df_temp["Cuts"] = df_temp["Cuts"].astype(str)
        # Concatenate df_temp with cutting_scheme
        cutting_schedule = pd.concat([cutting_schedule, df_temp], ignore_index=True)
    # Drop duplicate rows
    cutting_schedule = cutting_schedule.drop_duplicates()
    # Reset the index
    cutting_schedule = cutting_schedule.reset_index(drop=True)
    return cutting_schedule


def main() -> pd.DataFrame:
    """
    Executes the main workflow to calculate cutting schemes and generate a cutting schedule.
    """
    # Read the data from CSV
    data = pd.read_csv(CSV_URL)
    cutting_data = {}
    # Process data for each unique diameter
    for dia in data['Dia'].unique():
        data_dia = data[data['Dia'] == dia]
        # Calculate the cutting schemes and store them in a dictionary
        cutting_patterns = calculate_required_bars(data_dia, BAR_LENGTH)
        cutting_data[dia] = cutting_patterns
        # Print results
        print(f'Required number of reinforcement bars with diameter {dia} mm: {len(cutting_patterns)} pcs.')
        percent = ((data_dia['Qty'] * data_dia['Length']).sum()/(len(cutting_patterns) * BAR_LENGTH)) * 100
        print(f'Total scrap: {round(100 - percent, 2)}%')
    # Generate the cutting table
    cutting_schedule = generate_cutting_table(cutting_data)
    return cutting_schedule


if __name__ == "__main__":
    # Execute the main function and store the resulting cutting table
    cutting_schedule = main()


Required number of reinforcement bars with diameter 10 mm: 168 pcs.
Total scrap: 2.32%
Required number of reinforcement bars with diameter 12 mm: 138 pcs.
Total scrap: 1.47%
Required number of reinforcement bars with diameter 16 mm: 143 pcs.
Total scrap: 3.76%
Required number of reinforcement bars with diameter 20 mm: 759 pcs.
Total scrap: 4.82%


In [6]:
cutting_schedule_L = cutting_schedule.style.to_latex(hrules=True)

In [7]:
latex_table_to_png(cutting_schedule_L, 'cs')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>